In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
df_train = pd.read_csv('F:Pallavi/CodeSpyder/Data Sets/Titanic train.csv')
df_test = pd.read_csv('F:/Pallavi/CodeSpyder/Data Sets/Titanic test.csv')

In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
df_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
df_test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [7]:
df_train.isna().sum(axis=1).max()

2

In [8]:
col_num = list(df_train.select_dtypes(exclude='object').columns)

In [9]:
col_num

['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [10]:
col_num.remove('PassengerId')
col_num.remove('Survived')

In [11]:
col_num

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [12]:
col_cat = list(df_train.select_dtypes(include='object').columns)

In [13]:
col_cat

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [14]:
col_cat.remove('Name')
col_cat.remove('Ticket')
col_cat.remove('Cabin')

In [15]:
col_cat

['Sex', 'Embarked']

In [16]:
# new columns
# df_train['isCabin'] = (~df_train['Cabin'].isna()).astype(int)
# df_test['isCabin'] = (~df_test['Cabin'].isna()).astype(int)


# df_train['Title'] = df_train['Name'].apply(lambda name:name.split(',')[1].split('.')[0].strip())
# df_test['Title'] = df_test['Name'].apply(lambda name:name.split(',')[1].split('.')[0].strip())

# df_train['Selected_Titles'] = df_train['Title'].apply(lambda title: title if title in ['Mr','Miss','Mrs','Master','Dr','Rev'] else 'other')
# df_test['Selected_Titles'] = df_test['Title'].apply(lambda title: title if title in ['Mr','Miss','Mrs','Master','Dr','Rev'] else 'other')

In [17]:
# col_num.append('isCabin')
# col_cat.append('Selected_Titles')

In [18]:
col_final = col_num + col_cat
col_final

['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex', 'Embarked']

In [19]:
pipe_num = Pipeline([('imputer',SimpleImputer(strategy='median')),('scaler',MinMaxScaler())])

In [20]:
pipe_num

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', MinMaxScaler())])

In [21]:
pipe_cat = Pipeline([('imputer',SimpleImputer(strategy='most_frequent')),('encoder',OneHotEncoder())])

In [22]:
pipe_cat

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OneHotEncoder())])

In [23]:
preprocessor = ColumnTransformer([('num',pipe_num,col_num),('cat',pipe_cat,col_cat)])

In [24]:
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', MinMaxScaler())]),
                                 ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder', OneHotEncoder())]),
                                 ['Sex', 'Embarked'])])

In [25]:
final_pipe = Pipeline([('preprocessor',preprocessor),('model',RandomForestClassifier(n_estimators=20,max_depth=5))])

In [26]:
final_pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Embarked'])])),
                ('model',
                 RandomForestClassifier(max_depth=5, n_estimators=20))])

In [27]:
final_pipe.fit(df_train[col_final],df_train['Survived'])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Pclass', 'Age', 'SibSp',
                                                   'Parch', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  ['Sex', 'Embarked'])])),
                ('model',
                 RandomForestClassifier(max_depth=5, n_estimators=20))])

In [28]:
final_pipe.score(df_train[col_final],df_train.Survived)

0.8451178451178452

In [29]:
yp = final_pipe.predict(df_test[col_final])

In [30]:
submission = pd.DataFrame({'PassengerId':df_test['PassengerId'],'Survived':yp})

In [31]:
submission.to_csv('titanic_pipeline.csv',index=False)

In [32]:
import pickle

In [33]:
pickle.dump(final_pipe,open('titanic_pipeline.pkl','wb'))